Gather necessary pre-requisites:

In [1]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.fsl as fsl
import nipype.interfaces.spm as spm
import nibabel as nb
from nilearn.plotting import plot_anat
import numpy as np
import pandas as pd
# activate inline magics
%matplotlib inline 
import matplotlib.pyplot as plt
import os, re, json

In [2]:
# define base diectory for fMRI folder structure and FEDs
basedir="/fMRI/"
FED_dirs=[os.path.join(basedir, FED) 
          for FED in os.listdir(basedir)]

Get relevant files

In [3]:
# collect image files
files=sorted([os.path.join(basedir, subdir, content)
       for subdir in FED_dirs
       for content in os.listdir(subdir)
       if re.match(r'(.*.(nii|json))', content)])

In [4]:
# create a dictionary storing all files/subject
data={}
for fed in FED_dirs:
    ID=fed.rsplit('/', 1)[1]
    # define partition for each subject
    subject={"files":[], "parameters":{}}
    content=[]
    for file in files:
        if re.match(fr'(.*{ID}.*)', file):
            content.append(file)
    subject["files"].extend(content)
    data[ID]=subject
    
# create shortcut to FEDs
FEDs=[key for key in data.keys()]

In [5]:
# control results
print(data["FED023"])

{'files': ['/fMRI/FED023/FMRI_FS_0010_20141216144604_10.json', '/fMRI/FED023/FMRI_FS_0010_20141216144604_10.nii', '/fMRI/FED023/GRE_FIELD_MAPPING_0006_20141216144604_e1.json', '/fMRI/FED023/GRE_FIELD_MAPPING_0006_20141216144604_e1.nii', '/fMRI/FED023/GRE_FIELD_MAPPING_0006_20141216144604_e2.json', '/fMRI/FED023/GRE_FIELD_MAPPING_0006_20141216144604_e2.nii', '/fMRI/FED023/GRE_FIELD_MAPPING_0007_20141216144604_e2_ph.json', '/fMRI/FED023/GRE_FIELD_MAPPING_0007_20141216144604_e2_ph.nii', '/fMRI/FED023/GRE_FIELD_MAPPING_0021_20141216144604_e1.json', '/fMRI/FED023/GRE_FIELD_MAPPING_0021_20141216144604_e1.nii', '/fMRI/FED023/GRE_FIELD_MAPPING_0021_20141216144604_e2.json', '/fMRI/FED023/GRE_FIELD_MAPPING_0021_20141216144604_e2.nii', '/fMRI/FED023/GRE_FIELD_MAPPING_0022_20141216144604_e2_ph.json', '/fMRI/FED023/GRE_FIELD_MAPPING_0022_20141216144604_e2_ph.nii', '/fMRI/FED023/GRE_e1mag_bet_B.nii', '/fMRI/FED023/GRE_e1mag_bet_B_mask.nii', '/fMRI/FED023/GRE_fieldmap.nii', '/fMRI/FED023/T1_MPRAGE_00

Processing

Raw data artefact rejection (visual inspection)

decided against ICA, as it potentially introduces a lot of artifical signal into the data and is, after visual QC, redundant at this point.

Distortion correction \
(minimised with Scott @ scantime
-> still present in frontal poles etc.)

In [6]:
# read json info data for Echo times etc.
# define info of interest
GREspecs=["EchoTime"]
#fMRIspecs=[]
for subject in FEDs:
    # collect files/FED
    GREs=[file for file in data[subject]["files"]
           if re.match(r'(.*(_e1|_e2(?!_ph)).*.json)', file)]
#    funcs=[file for file in data[subject]["files"]
#           if re.match(r'(.*(FMRI).*.json)', file)]

    # collect short and long TE from the respective echoes
    gre=[]
    for file in GREs:
        with open(file) as json_file:
            info=json.load(json_file)
            gre.extend([info[param] for param in GREspecs])
    # if there is more than one echo set, take the first one as they are likely to be closer to the fMRI acquisition
    if len(gre) > 2:
        print(subject, "\n", "More than 2 GRE_FIELD sequences",
              {len(gre)}, ".... selecting first two for TE extraction", "\n")
        gre = gre[0:2]
    # put TEs into data
    data[subject]["parameters"]["grefield_TEs"]=gre

FED022 
 More than 2 GRE_FIELD sequences {4} .... selecting first two for TE extraction 

FED023 
 More than 2 GRE_FIELD sequences {4} .... selecting first two for TE extraction 



In [7]:
# control data

# There seem to be several more omissions than expected
# investigate
GRE_excluded=[]
for sub in FEDs:
    print(sub, "\n", data[sub]["parameters"]["grefield_TEs"])
    if len(data[sub]["parameters"]["grefield_TEs"]) < 2:
        print("Not enough GRE_FIELD sequences to read out .... investigate", "\n")
        GRE_excluded.append(sub)
    # can't be larger than 2 ... see above^^
    else:
        print(len(data[sub]["parameters"]["grefield_TEs"]),
              "GRE_FIELD sequences", " \n")
        
# exclude FEDs from distortion correction
print("\n\n", "The following subjects where excluded from distortion correction due to insufficient data: ",
      "\n", sorted(GRE_excluded))

# make new list for calculating fieldmaps
gre_FEDs=[file for file in FEDs if file not in GRE_excluded]

FED015 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED012 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED047 
 []
Not enough GRE_FIELD sequences to read out .... investigate 

FED037 
 []
Not enough GRE_FIELD sequences to read out .... investigate 

FED030 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED006 
 []
Not enough GRE_FIELD sequences to read out .... investigate 

FED054 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED042 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED024 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED038 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED045 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED061 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED035 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED022 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED009 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED036 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED019 
 [0.00492, 0.00738]
2 GRE_FIELD sequences  

FED017 
 [0.00492, 0.00738]
2 GRE_FI

In [14]:
# prepare the gradient field scans (1 phase and 1 magnitude image)
# for usage in distortion correction: fieldmap FSL, rest SPM

# BET on magnitude images
magnit=[]
for sub in gre_FEDs:
    # collect magnitude images
    for i in data[sub]["files"]:
        if re.match(r'.*(_e1).*.nii', i):
            magnit.append(i)

# perform brain extraction
for file in magnit:
    bet=Node(fsl.BET(), name="bet")
    bet.inputs.in_file=file
    bet.inputs.out_file=f"{file.split('_', 1)[0]}_e1mag_bet_B.nii"
    bet.inputs.reduce_bias= True
    bet.inputs.output_type="NIFTI"
    res=bet.run()
    print(res.outputs)
    # plot before after comparison
    plot_anat(bet.inputs.in_file, title='BET input', cut_coords=(10,10,10),
          display_mode='ortho', dim=-1, draw_cross=False, annotate=False);
    plot_anat(res.outputs.out_file, title='BET output', cut_coords=(10,10,10),
          display_mode='ortho', dim=-1, draw_cross=False, annotate=False);

FIELD_MAPPING_0006_20141202145526_e1.json


In [10]:
# build the fieldmaps
for sub in gre_FEDs:
    # collect relevant data
    for i in data[sub]["files"]:
        if re.match(r'.*(FMRI).*.nii', i):
            func=i
        elif re.match(r'.*(_ph).*.nii', i):
            phase=i
        elif re.match(r'.*(_e1mag_bet).*(?<!_mask).nii', i):
            mag=i
    # get delta TE (ms)
    deltaTE=(data[sub]["parameters"]["grefield_TEs"][1] - data[sub]["parameters"]["grefield_TEs"][0])*1000
    # print all parameters
    print(func,"\n", mag,"\n", phase,"\n", deltaTE)
    # call FSL interface to calculate fieldmaps
    build_fieldmap=Node(fsl.PrepareFieldmap(), name='GRE_fieldmap')
    build_fieldmap.inputs.in_phase= phase
    build_fieldmap.inputs.in_magnitude= mag
    build_fieldmap.inputs.delta_TE= deltaTE
    build_fieldmap.inputs.out_fieldmap=f"{os.path.join(basedir, sub)}/GRE_fieldmap.nii"
    build_fieldmap.inputs.output_type="NIFTI"
    # run it
    results= build_fieldmap.run()
    # print output
    print(results.outputs)

/fMRI/FED015/FMRI_FS_0010_20141202145526_10.nii 
 /fMRI/FED015/GRE_e1mag_bet_B.nii 
 /fMRI/FED015/GRE_FIELD_MAPPING_0007_20141202145526_e2_ph.nii 
 2.4600000000000004
200915-09:17:53,793 nipype.workflow INFO:
	 [Node] Setting-up "GRE_fieldmap" in "/tmp/tmp73euhz3r/GRE_fieldmap".
200915-09:17:53,797 nipype.workflow INFO:
	 [Node] Running "GRE_fieldmap" ("nipype.interfaces.fsl.epi.PrepareFieldmap"), a CommandLine Interface with command:
fsl_prepare_fieldmap SIEMENS /fMRI/FED015/GRE_FIELD_MAPPING_0007_20141202145526_e2_ph.nii /fMRI/FED015/GRE_e1mag_bet_B.nii /fMRI/FED015/GRE_fieldmap.nii 2.460000
200915-09:17:53,970 nipype.interface INFO:
	 stdout 2020-09-15T09:17:53.970512:Loading volumes
200915-09:17:53,973 nipype.interface INFO:
	 stdout 2020-09-15T09:17:53.973781:Phase loaded
200915-09:17:53,974 nipype.interface INFO:
	 stdout 2020-09-15T09:17:53.973781:Magnitude loaded
200915-09:17:53,976 nipype.interface INFO:
	 stdout 2020-09-15T09:17:53.973781:Mask loaded
200915-09:17:53,978 nipyp

Motion correct / realign

Slicetime correction (?)

I will omit this step as advised by Poldrack, Nichols and Mumford "Handbook of fMRI data analysis"(2011)\
TR= 1800ms  > 2seconds, interleaved acquisition\
event related analysis is relatively robust to slice-timing problems here.

Spatial normalisation

Spatial Smoothing